<a href="https://colab.research.google.com/github/realnribal/frankenstein-finetune/blob/main/qlora_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Configuration de l'environnement

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Configuration
PROJECT_DIR = '/content/drive/MyDrive/00-Finetuning'
MODULES_DIR = f'{PROJECT_DIR}/colab_modules'
DATA_PATH = f'{PROJECT_DIR}/frankenstein_chunks.csv'
REQUIREMENTS_FILE = f'{MODULES_DIR}/requirements.txt'
REQUIRED_PACKAGES = ['datasets', 'bitsandbytes', 'transformers', 'peft', 'accelerate', 'dill']

import os
import importlib
import subprocess

# Vérifier si Google Drive est monté
try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        print("Montage de Google Drive...")
        drive.mount('/content/drive')
    else:
        print("Google Drive déjà monté")
except ImportError:
    print("Ce code est conçu pour s'exécuter dans Google Colab")

# Créer le répertoire des modules si nécessaire
if not os.path.exists(MODULES_DIR):
    os.makedirs(MODULES_DIR)
    print(f"Création du répertoire {MODULES_DIR}")

# Mettre à jour ou créer le fichier requirements.txt
with open(REQUIREMENTS_FILE, 'w') as f:
    for package in REQUIRED_PACKAGES:
        f.write(f"{package}\n")
print(f"Fichier requirements.txt créé/mis à jour dans {REQUIREMENTS_FILE}")

# Ajouter le répertoire des modules au chemin Python
import sys
if MODULES_DIR not in sys.path:
    sys.path.append(MODULES_DIR)
    print(f"Répertoire {MODULES_DIR} ajouté au chemin Python")

# Fonction pour vérifier si un paquet est installé
def is_package_installed(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        # Vérifier aussi si le paquet est présent dans le répertoire des modules
        package_dir = os.path.join(MODULES_DIR, package_name)
        if os.path.exists(package_dir):
            return True
        return False

# Vérifier quels paquets doivent être installés
packages_to_install = []
for package in REQUIRED_PACKAGES:
    if not is_package_installed(package):
        packages_to_install.append(package)
        print(f"Le paquet {package} n'est pas installé et sera ajouté")
    else:
        print(f"Le paquet {package} est déjà installé")

# N'installer que les paquets manquants
if packages_to_install:
    print(f"Installation des paquets manquants dans {MODULES_DIR}...")
    packages_str = " ".join(packages_to_install)
    !pip install {packages_str} -t {MODULES_DIR}
    print("Installation terminée.")
else:
    print("Tous les paquets requis sont déjà installés. Aucune installation nécessaire.")

Google Drive déjà monté
Fichier requirements.txt créé/mis à jour dans /content/drive/MyDrive/00-Finetuning/colab_modules/requirements.txt
Répertoire /content/drive/MyDrive/00-Finetuning/colab_modules ajouté au chemin Python
Le paquet datasets est déjà installé
Le paquet bitsandbytes est déjà installé
Le paquet transformers est déjà installé
Le paquet peft est déjà installé
Le paquet accelerate est déjà installé
Le paquet dill est déjà installé
Tous les paquets requis sont déjà installés. Aucune installation nécessaire.


In [3]:
import random
import torch
import pandas as pd
from datasets import Dataset
import peft
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)

set_seed()

### 1. Definir le model

In [16]:
mistral7b = 'mistralai/Mistral-7B-v0.1'
model_name = mistral7b

### 2. EDA

In [6]:
df = pd.read_csv(DATA_PATH)
df.head()

,text
0,﻿The Project Gutenberg eBook of Frankenstein; ...
1,Further corrections by Menno de Leeuw.\n\n\n**...
2,"I am already far north of London, and as I wal..."
3,Its productions and features may be without ex...
4,But supposing all these conjectures to be fals...


## 3. Inspection des données

In [7]:
print("Dataframe Info:")
print(df.info())
print("\n")
print("Dataframe Description:")
print(df.describe())
print("\n")
print("Number of unique values in each column:")
print(df.nunique())
random_index= random.randint(0, len(df) - 1)
df.loc[random_index, 'text']

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    481 non-null    object
dtypes: object(1)
memory usage: 3.9+ KB
None


Dataframe Description:
                                                     text
count                                                 481
unique                                                481
top     International donations are gratefully accepte...
freq                                                    1


Number of unique values in each column:
text    481
dtype: int64


'The thatch had fallen in, the walls were unplastered, and the\ndoor was off its hinges. I ordered it to be repaired, bought some\nfurniture, and took possession, an incident which would doubtless have\noccasioned some surprise had not all the senses of the cottagers been\nbenumbed by want and squalid poverty. As it was, I lived ungazed at\nand unmolested, hardly thanked for the pittance of food and clothes\nwhich I gave, so much does suffering blunt even the coarsest sensations\nof men.\n\nIn this retreat I devoted the morning to labour; but in the evening,\nwhen the weather permitted, I walked on the stony beach of the sea to\nlisten to the waves as they roared and dashed at my feet. It was a\nmonotonous yet ever-changing scene. I thought of Switzerland; it was\nfar different from this desolate and appalling landscape. '

In [8]:
df.isnull().sum()

,0
text,0


### 4. Création d'un split train/test

In [10]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

### 5. Conversion des DataFrames en datasets Hugging Face

In [11]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

### 6. Quantification du modèle

Comment configurer le token
```python
  # Au début de votre notebook
import os
from google.colab import userdata

# Récupérer le token depuis les secrets
hf_token = userdata.get('HUGGING_FACE_TOKEN')

# Configurer le token pour Hugging Face
os.environ["HUGGING_FACE_HUB_TOKEN"] = hf_token

# Vérifier que le token est bien configuré (optionnel)
print("Token configuré:", "✓" if hf_token else "✗")
```

> Il est nécessaire de générer untoken pour avoir accès sur Hugging Face

In [21]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

### 7. Charger le modèle


In [22]:
quant_config = BitsAndBytesConfig(

    load_in_4bit=True,             # Devrait être True pour activer la quantification 4-bit
    bnb_4bit_use_double_quant=True, # Devrait être True/False pour activer ou non la double quantification
    bnb_4bit_quant_type='nf4',      # Devrait être 'nf4' ou 'fp4' pour définir le type de quantification
    bnb_4bit_compute_dtype=torch.bfloat16    # Devrait être torch.float16/torch.bfloat16 pour définir le type de calcul
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config)
print("\n\nModel is running on:" + str(model.device) + "\n")
model

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]



Model is running on:cuda:0



MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

> Ce code charge un modèle de langage causal à partir de Hugging Face et affiche sur quel appareil il est exécuté. En détail :
1. `AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config)`
   - Charge un modèle de langage causal (comme Mistral, Llama, etc.) depuis Hugging Face
   - `model_name` est une variable qui contient l'identifiant du modèle (par exemple "mistralai/Mistral-7B-v0.1")
   - `quantization_config=quant_config` applique la configuration de quantification définie préalablement avec BitsAndBytes pour réduire l'empreinte mémoire du modèle (probablement en 4-bit comme vu précédemment)
2. `print("\n\nModel is running on:" + str(model.device) + "\n")`
   - Affiche l'appareil sur lequel le modèle est chargé
   - `model.device` indique si le modèle est sur CPU ou GPU (par exemple "cuda:0" pour le premier GPU)
   - Cette information est utile pour vérifier que le modèle est bien chargé sur le bon appareil de calcul


### 8. fine-tuning avec QLoRA

#### 8.1 Import des modules nécessaires de PEFT (Parameter-Efficient Fine-Tuning)

In [23]:
# Import des modules nécessaires dont : PEFT (Parameter-Efficient Fine-Tuning)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

#### 8.2 Préparation du modèle pour QLoRA

In [26]:
# Préparation du modèle pour QLoRA
model = prepare_model_for_kbit_training(model)


#### 8.3 Configuration de LoRA

In [28]:
# Configuration de LoRA
config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    task_type='CAUSAL_LM'
)

> Définit les hyperparamètres LoRA qui contrôlent comment le modèle sera fine-tuné

#### 8.4 Application de LoRA au modèle

In [29]:
model = get_peft_model(model, config)

> Transforme le modèle de base en un modèle compatible PEFT avec les adaptateurs LoRA

#### 8.5 Chargement du tokenizer

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

> Charge le tokenizer correspondant au modèle

#### 8.6 Tokenisation des datasets

In [35]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenized_train_dataset = train_dataset.map(lambda examples: tokenizer(examples["text"], padding="longest", truncation=True), batched=True)
tokenized_test_dataset = test_dataset.map(lambda examples: tokenizer(examples["text"], padding="longest", truncation=True), batched=True)

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

Map:   0%|          | 0/97 [00:00<?, ? examples/s]